In [1]:
# 首先我们定义模型、数据与相应的参数。
import tensorflow as tf

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=[])

# 然后我们定义相应的 TensorBoard 日志目录，同时对模型使用 fit() 进行训练：
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs')

model.fit(x=x_train, y=y_train, 
          epochs=3, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])


11493376/11490434 [==============================] - 2s 0us/step


In [2]:
tensorboard --logdir logs

SyntaxError: invalid syntax (2505290755.py, line 1)

## 2. 如何在 TensorBoard 之中生成使用 tf.function 函数定义的图(这个很实用)
    在实际的应用过程之中，有很多的情况下，我们需要使用 tf.function 来加速模型的速度并自定义训练过程。那么这个时候我们要如何才能查看网络的模型结构图呢？
    其实也很简单，我们只需经过如下几个步骤：
    a.确保 tf.function 函数修饰了我们需要进行可视化的操作，这边就是模型的过程；
    b.创建一个 TensorBoard 的日志写入器 tf.summary.create_file_writer() ；
    c.通过 tf.summary.trace_on() API 进行变量路径的追踪；
    d,执行我们需要可视化的操作；
    e.使用 tf.summary.trace_export() API 将图写入日志。
    在这里，我们可以使用一个很简单的例子来查看操作的结构：

In [ ]:

# 定义网络的操作
@tf.function
def test_func(x, y):
  z = tf.matmul(x, y)
  z = z * 5.0
  z = tf.nn.relu(z)
  return z

# 创建写入器
writer = tf.summary.create_file_writer('./logs/3')

# 创建初试数据
x = tf.random.uniform((5, 5))
y = tf.random.uniform((5, 5))

# 开启变量追踪
tf.summary.trace_on(graph=True, profiler=True)

# 运行程序
z = test_func(x, y)

# 将日志输出
with writer.as_default():
  tf.summary.trace_export(
      name="test_func_graph",
      step=1,
      profiler_outdir='./logs/3')
